<a href="https://colab.research.google.com/github/SuminBae97/PytorchStudy/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 64 kB 913 kB/s 
     |████████████████████████████████| 1.2 MB 9.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [2]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [3]:
import spacy
spacy_en = spacy.load('en')
spacy_de = spacy.load('de')


In [4]:
tokenized = spacy_en.tokenizer('i am a graduate student')

for i,token in enumerate(tokenized):
    print(f'index : {i}: {token.text}')

index : 0: i
index : 1: am
index : 2: a
index : 3: graduate
index : 4: student


In [5]:
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]    

In [6]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [7]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:03<00:00, 302kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 92.7kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 88.1kB/s]


In [14]:
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])


['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [15]:
for i in range(3):
    print(vars(train_dataset.examples[i])['src'])
    print(vars(train_dataset.examples[i])['trg'])



['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']
['mehrere', 'männer', 'mit', 'schutzhelmen', 'bedienen', 'ein', 'antriebsradsystem', '.']
['several', 'men', 'in', 'hard', 'hats', 'are', 'operating', 'a', 'giant', 'pulley', 'system', '.']
['ein', 'kleines', 'mädchen', 'klettert', 'in', 'ein', 'spielhaus', 'aus', 'holz', '.']
['a', 'little', 'girl', 'climbing', 'into', 'a', 'wooden', 'playhouse', '.']


In [16]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset,min_freq=2)

print(f'len(src) : {len(SRC.vocab)}')
print(f'len(trg) : {len(TRG.vocab)}')

len(src) : 7855
len(trg) : 5893


In [17]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator ,test_iterator = BucketIterator.splits(
    (train_dataset,valid_dataset,test_dataset),
    batch_size=BATCH_SIZE,
    device=device
)




In [20]:
list(train_iterator)[0]


[torchtext.data.batch.Batch of size 128 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 128x29 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 128x28 (GPU 0)]

In [ ]:
j=0
src_1=None
trg_1=None
i=0
for i, batch in enumerate(train_iterator):
    src = batch.src
    src_1 = src
    trg = batch.trg
    trg_1 = trg

    print(f"{i} 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    # for i in range(src.shape[1]):
    #     print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

    # 첫 번째 배치만 확인
    
    

In [69]:

for i in range(len(trg_1)):
    for j in range(len(trg_1[i])):
         print(TRG.vocab.itos[trg_1[i][j]],end=' ')
    print()
    print("-----------------------------------------------------------")     




<sos> two people are playing guitar , while a third is <unk> the drums . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
-----------------------------------------------------------
<sos> a woman wearing a brown costume and fuzzy black wings is riding a bike <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
-----------------------------------------------------------
<sos> a very <unk> looking woman is holding a pot and some branches above her head . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
-----------------------------------------------------------
<sos> a group of people on a dock by a city boarding various boats <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
-----------------------------------------------------------
<sos> a man is yelling at someone from a game booth . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
---------------------------

In [49]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']